# Lectura del dataset

##### Paso 1: Lectura del dataset "Clinton/Text-to-sql-v1"

In [65]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")

import datasets

dataset = datasets.load_dataset("Clinton/Text-to-sql-v1")
df = dataset["train"].to_pandas()

df = df[df['source'].isin(['sql_create_context', 'spider', 'mimicsql_data'])]
df.head()

df_copy = df.copy()


def add_length_column(df):
    df["total_length"] = 0
    for column_name in ["instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

def filter_by_join_count(df, difficulty, number_of_samples):
    """Filtrado de ejemplos del dataset por cantidad de JOINS en el ejemplo"""
    if difficulty == "lvl1":
        join_mask = df["response"].str.count("JOIN") == 0
    elif difficulty == "lvl2":
        join_mask = df["response"].str.count("JOIN") == 1
    elif difficulty == "lvl3":
        join_mask = df["response"].str.count("JOIN") == 2
    elif difficulty == "lvl4":
        join_mask = df["response"].str.count("JOIN") >= 3
    else:
        raise ValueError("Invalid difficulty level")

    join_df = df[join_mask]
    return join_df.sample(n=number_of_samples, random_state=1)

df_copy = add_length_column(df_copy)
df_copy.head()

,instruction,input,response,source,text,total_length
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...,sql_create_context,Below are sql tables schemas paired with instr...,25
3,List the record of 0-1 from the table?,CREATE TABLE table_14656147_2 (\n week VARC...,SELECT week FROM table_14656147_2 WHERE record...,sql_create_context,Below are sql tables schemas paired with instr...,25
4,"Which silver has a Gold smaller than 12, a Ran...",CREATE TABLE table_name_24 (\n silver VARCH...,SELECT silver FROM table_name_24 WHERE gold < ...,sql_create_context,Below are sql tables schemas paired with instr...,47
7,Name the most 3 credits,CREATE TABLE table_148535_2 (\n Id VARCHAR\n),SELECT MIN(3 AS _credits) FROM table_148535_2,sql_create_context,Below are sql tables schemas paired with instr...,18
9,"In what years was there a rank lower than 9, u...",CREATE TABLE table_name_63 (\n years VARCHA...,SELECT years FROM table_name_63 WHERE matches ...,sql_create_context,Below are sql tables schemas paired with instr...,47


##### Paso 2: Filtrado del dataset por cantidad de joins en los ejemplos

In [2]:
lvl1_df = filter_by_join_count(df_copy, "lvl1", 25)
lvl2_df = filter_by_join_count(df_copy, "lvl2", 25)
lvl3_df = filter_by_join_count(df_copy, "lvl3", 25)
lvl4_df = filter_by_join_count(df_copy, "lvl4", 25)

# Buscamos las respuestas (posteriormente usadas para generar los joins con chatgpt)
responses_lvl1 = lvl1_df["response"].tolist()
responses_lvl2 = lvl2_df["response"].tolist()
responses_lvl3 = lvl3_df["response"].tolist()
responses_lvl4 = lvl4_df["response"].tolist()
# Buscamos los inputs (posteriormente usadas para generar los comentarios con chatgpt)
context_lvl1 = lvl1_df["input"].tolist()
context_lvl2 = lvl2_df["input"].tolist()
context_lvl3 = lvl3_df["input"].tolist()
context_lvl4 = lvl4_df["input"].tolist()

# Generación de relaciones y comentarios con chatgpt
##### Como no tenemos data con comentarios o relaciones entre tablas, usamos chatgpt para poder generarlos y agregarlos al set de datos

#### Paso 1: Generamos el prompt para chatgpt
##### Un prompt para relaciones y otro para comentarios

In [9]:
from train.mistral7bbase.functions.data_functions import get_gpt_relationships_prompt
import pyperclip

p = get_gpt_relationships_prompt(responses_lvl2[-5:])
pyperclip.copy(p)
# print(p)

In [18]:
from train.mistral7bbase.functions.data_functions import get_gpt_comments_prompt
import pyperclip

p = get_gpt_comments_prompt(context_lvl2[:1])
p = get_gpt_comments_prompt(context_lvl2[1:3])
p = get_gpt_comments_prompt(context_lvl2[3:6])
p = get_gpt_comments_prompt(context_lvl2[6:10])
p = get_gpt_comments_prompt(context_lvl2[10:15])
p = get_gpt_comments_prompt(context_lvl2[15:20])

p = get_gpt_comments_prompt(context_lvl4[-5:])
pyperclip.copy(p)
# print(p)

In [27]:
#####################
relations_lvl1 = [
'-- No JOIN relationships found',
'-- No JOIN relationships found',
'-- No JOIN relationships found',
'-- No JOIN relationships found',
'-- No JOIN relationships found',]
relations_lvl2 = [
'-- playlist_tracks.playlist_id can be joined with playlists.id',
'-- teachers.teacher_id can be joined with detention.teacher_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- club.club_id can be joined with player.club_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',]
relations_lvl3 = [
'-- trained_in.physician can be joined with physician.employeeid, -- trained_in.treatment can be joined with procedures.code',
'-- journal_committee.Editor_ID can be joined with editor.Editor_ID, -- journal_committee.Journal_ID can be joined with journal.Journal_ID',
'-- authors.authid can be joined with authorship.authid, -- authorship.paperid can be joined with papers.paperid',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- loan.branch_id can be joined with bank.branch_id, -- loan.cust_id can be joined with customer.cust_id',]
relations_lvl4 = [
'-- problems.product_id can be joined with product.product_id, -- problems.reported_by_staff_id can be joined with staff.staff_id',
'-- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid',
'-- CLASS.prof_num can be joined with employee.emp_num, -- employee.emp_num can be joined with professor.emp_num, -- professor.dept_code can be joined with department.dept_code',
'-- customers.customer_id can be joined with customer_orders.customer_id, -- customer_orders.order_id can be joined with order_items.order_id, -- order_items.product_id can be joined with products.product_id',
'-- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid'
]


In [41]:
context_lvl1 = [
    '''
    CREATE TABLE IF NOT EXISTS table_name_65 (
        away_captain VARCHAR, -- Name of the away team's captain
        home_captain VARCHAR, -- Name of the home team's captain
        result VARCHAR -- Match result
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_21 (
        place VARCHAR, -- Name of the place
        country VARCHAR -- Name of the country
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        code number, -- Unique code for the procedure
        name text, -- Name of the procedure
        cost number -- Cost of the procedure
    );

    CREATE TABLE IF NOT EXISTS nurse (
        employeeid number, -- Unique ID for the nurse
        name text, -- Name of the nurse
        position text, -- Position of the nurse
        registered boolean, -- Whether the nurse is registered or not
        ssn number -- Social security number of the nurse
    );

    CREATE TABLE IF NOT EXISTS on_call (
        nurse number, -- ID of the nurse on call
        blockfloor number, -- Floor of the block
        blockcode number, -- Code of the block
        oncallstart time, -- Start time of the on-call
        oncallend time -- End time of the on-call
    );

    CREATE TABLE IF NOT EXISTS stay (
        stayid number, -- Unique ID for the stay
        patient number, -- ID of the patient
        room number, -- Number of the room
        staystart time, -- Start time of the stay
        stayend time -- End time of the stay
    );

    CREATE TABLE IF NOT EXISTS appointment (
        appointmentid number, -- Unique ID for the appointment
        patient number, -- ID of the patient
        prepnurse number, -- ID of the nurse who prepares the patient
        physician number, -- ID of the physician
        start time, -- Start time of the appointment
        end time, -- End time of the appointment
        examinationroom text -- Name of the examination room
    );

    CREATE TABLE IF NOT EXISTS trained_in (
        physician number, -- ID of the physician
        treatment number, -- ID of the treatment
        certificationdate time, -- Date of the certification
        certificationexpires time -- Date of the certification expiration
    );

    CREATE TABLE IF NOT EXISTS physician (
        employeeid number, -- Unique ID for the physician
        name text, -- Name of the physician
        position text, -- Position of the physician
        ssn number -- Social security number of the physician
    );

    CREATE TABLE IF NOT EXISTS undergoes (
        patient number, -- ID of the patient
        procedures number, -- ID of the procedure
        stay number, -- ID of the stay
        dateundergoes time, -- Date of the procedure
        physician number, -- ID of the physician
        assistingnurse number -- ID of the assisting nurse
    );

    CREATE TABLE IF NOT EXISTS prescribes (
        physician number, -- ID of the physician
        patient number, -- ID of the patient
        medication number, -- ID of the medication
        date time, -- Date of the prescription
        appointment number, -- ID of the appointment
        dose text -- Dose of the medication
    );

    CREATE TABLE IF NOT EXISTS department (
        departmentid number, -- Unique ID for the department
        name text, -- Name of the department
        head number -- ID of the department head
    );

    CREATE TABLE IF NOT EXISTS medication (
        code number, -- Unique code for the medication
        name text, -- Name of the medication
        brand text, -- Brand of the medication
        description text -- Description of the medication
    );

    CREATE TABLE IF NOT EXISTS block (
        blockfloor number, -- Floor of the block
        blockcode number -- Code of the block
    );

    CREATE TABLE IF NOT EXISTS patient (
        ssn number, -- Unique ID for the patient
        name text, -- Name of the patient
        address text, -- Address of the patient
        phone text, -- Phone number of the patient
        insuranceid number, -- ID of the insurance
        pcp number -- ID of the primary care physician
    );

    CREATE TABLE IF NOT EXISTS room (
        roomnumber number, -- Number of the room
        roomtype text, -- Type of the room
        blockfloor number, -- Floor of the block
        blockcode number, -- Code of the block
        unavailable boolean -- Whether the room is available or not
    );

    CREATE TABLE IF NOT EXISTS affiliated_with (
        physician number, -- ID of the physician
        department number, -- ID of the department
        primaryaffiliation boolean -- Whether the affiliation is primary or not
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_84 (
        gain INTEGER, -- Amount of gain
        long VARCHAR, -- Longitude
        name VARCHAR, -- Name
        loss VARCHAR, -- Amount of loss
        avg_g VARCHAR -- Average gain
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_69 (
        date VARCHAR, -- Date of the competition
        competition VARCHAR, -- Name of the competition
        result VARCHAR -- Competition result
    );
    '''
]

context_lvl2 = [
    '''
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        playlist_id VARCHAR, -- Unique identifier for the playlist
        track_id VARCHAR -- Unique identifier for the track
    );

    CREATE TABLE IF NOT EXISTS playlists (
        name VARCHAR, -- Name of the playlist
        id VARCHAR -- Unique identifier for the playlist
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS students_in_detention (
        student_id number, -- Unique identifier for the student
        detention_id number, -- Unique identifier for the detention
        incident_id number -- Unique identifier for the incident
    );

    CREATE TABLE IF NOT EXISTS student_addresses (
        student_id number, -- Unique identifier for the student
        address_id number, -- Unique identifier for the address
        date_address_from time, -- Date when the student started living at this address
        date_address_to time, -- Date when the student stopped living at this address
        monthly_rental number, -- Amount of monthly rental for the address
        other_details text -- Any other details about the address
    );

    CREATE TABLE IF NOT EXISTS assessment_notes (
        notes_id number, -- Unique identifier for the notes
        student_id number, -- Unique identifier for the student
        teacher_id number, -- Unique identifier for the teacher
        date_of_notes time, -- Date when the notes were written
        text_of_notes text, -- The actual text of the notes
        other_details text -- Any other details about the notes
    );

    CREATE TABLE IF NOT EXISTS students (
        student_id number, -- Unique identifier for the student
        address_id number, -- Unique identifier for the address
        first_name text, -- First name of the student
        middle_name text, -- Middle name of the student
        last_name text, -- Last name of the student
        cell_mobile_number text, -- Cell/mobile number of the student
        email_address text, -- Email address of the student
        date_first_rental time, -- Date when the student first rented a property
        date_left_university time, -- Date when the student left the university
        other_student_details text -- Any other details about the student
    );

    CREATE TABLE IF NOT EXISTS addresses (
        address_id number, -- Unique identifier for the address
        line_1 text, -- First line of the address
        line_2 text, -- Second line of the address
        line_3 text, -- Third line of the address
        city text, -- City of the address
        zip_postcode text, -- Zip/postal code of the address
        state_province_county text, -- State/province/county of the address
        country text, -- Country of the address
        other_address_details text -- Any other details about the address
    );

    CREATE TABLE IF NOT EXISTS behavior_incident (
        incident_id number, -- Unique identifier for the incident
        incident_type_code text, -- Code for the type of incident
        student_id number, -- Unique identifier for the student
        date_incident_start time, -- Date when the incident started
        date_incident_end time, -- Date when the incident ended
        incident_summary text, -- Summary of the incident
        recommendations text, -- Recommendations for the incident
        other_details text -- Any other details about the incident
    );

    CREATE TABLE IF NOT EXISTS teachers (
        teacher_id number, -- Unique identifier for the teacher
        address_id number, -- Unique identifier for the address
        first_name text, -- First name of the teacher
        middle_name text, -- Middle name of the teacher
        last_name text, -- Last name of the teacher
        gender text, -- Gender of the teacher
        cell_mobile_number text, -- Cell/mobile number of the teacher
        email_address text, -- Email address of the teacher
        other_details text -- Any other details about the teacher
    );

    CREATE TABLE IF NOT EXISTS ref_incident_type (
        incident_type_code text, -- Code for the type of incident
        incident_type_description text -- Description of the type of incident
    );

    CREATE TABLE IF NOT EXISTS detention (
        detention_id number, -- Unique identifier for the detention
        detention_type_code text, -- Code for the type of detention
        teacher_id number, -- Unique identifier for the teacher
        datetime_detention_start time, -- Date and time when the detention started
        datetime_detention_end time, -- Date and time when the detention ended
        detention_summary text, -- Summary of the detention
        other_details text -- Any other details about the detention
    );

    CREATE TABLE IF NOT EXISTS ref_address_types (
        address_type_code text, -- Code for the type of address
        address_type_description text -- Description of the type of address
    );

    CREATE TABLE IF NOT EXISTS ref_detention_type (
        detention_type_code text, -- Code for the type of detention
        detention_type_description text -- Description of the type of detention
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icustay_id text, -- Unique identifier for the ICU stay
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug prescribed
        formulary_drug_cd text, -- Code for the drug in the formulary
        route text, -- Route of administration for the drug
        drug_dose text -- Dose of the drug prescribed
    );

    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icd9_code text, -- Code for the procedure in the ICD-9 classification
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );

    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icd9_code text, -- Code for the diagnosis in the ICD-9 classification
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );

    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        itemid text, -- Unique identifier for the lab item
        charttime text, -- Date and time of the lab result
        flag text, -- Flag indicating the status of the lab result
        value_unit text, -- Unit of measurement for the lab result
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed in the hospital
        insurance text, -- Type of insurance
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis of the subject
        dod text, -- Date of death of the subject
        dob_year text, -- Year of birth of the subject
        dod_year text, -- Year of death of the subject
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS club (
        club_id number, -- Unique identifier for the club
        name text, -- Name of the club
        region text, -- Region of the club
        start_year text -- Year the club was founded
    );

    CREATE TABLE IF NOT EXISTS competition (
        competition_id number, -- Unique identifier for the competition
        year number, -- Year of the competition
        competition_type text, -- Type of competition
        country text -- Country where the competition took place
    );

    CREATE TABLE IF NOT EXISTS player (
        player_id number, -- Unique identifier for the player
        name text, -- Name of the player
        position text, -- Position of the player
        club_id number, -- Unique identifier for the club
        apps number, -- Number of apps played by the player
        tries number, -- Number of tries scored by the player
        goals text, -- Number of goals scored by the player
        points number -- Number of points scored by the player
    );

    CREATE TABLE IF NOT EXISTS club_rank (
        rank number, -- Rank of the club
        club_id number, -- Unique identifier for the club
        gold number, -- Number of gold medals won by the club
        silver number, -- Number of silver medals won by the club
        bronze number, -- Number of bronze medals won by the club
        total number -- Total number of medals won by the club
    );

    CREATE TABLE IF NOT EXISTS competition_result (
        competition_id number, -- Unique identifier for the competition
        club_id_1 number, -- Unique identifier for the first club
        club_id_2 number, -- Unique identifier for the second club
        score text -- Score of the competition
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icd9_code text, -- Code for the procedure in the ICD-9 classification
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );

    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icd9_code text, -- Code for the diagnosis in the ICD-9 classification
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );

    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        icustay_id text, -- Unique identifier for the ICU stay
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug prescribed
        formulary_drug_cd text, -- Code for the drug in the formulary
        route text, -- Route of administration for the drug
        drug_dose text -- Dose of the drug prescribed
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed in the hospital
        insurance text, -- Type of insurance
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis of the subject
        dod text, -- Date of death of the subject
        dob_year text, -- Year of birth of the subject
        dod_year text, -- Year of death of the subject
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );

    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Unique identifier for the hospital admission
        itemid text, -- Unique identifier for the lab item
        charttime text, -- Date and time of the lab result
        flag text, -- Flag indicating the status of the lab result
        value_unit text, -- Unit of measurement for the lab result
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );
    '''
]

context_lvl3 = [
    '''
    CREATE TABLE IF NOT EXISTS block (
        blockfloor number, -- Floor number of the block
        blockcode number -- Code of the block
    );

    CREATE TABLE IF NOT EXISTS on_call (
        nurse number, -- ID of the nurse on call
        blockfloor number, -- Floor number of the block
        blockcode number, -- Code of the block
        oncallstart time, -- Start time of the on-call shift
        oncallend time -- End time of the on-call shift
    );

    CREATE TABLE IF NOT EXISTS undergoes (
        patient number, -- ID of the patient
        procedures number, -- ID of the procedure
        stay number, -- ID of the stay
        dateundergoes time, -- Date of the procedure
        physician number, -- ID of the physician
        assistingnurse number -- ID of the assisting nurse
    );

    CREATE TABLE IF NOT EXISTS medication (
        code number, -- Code of the medication
        name text, -- Name of the medication
        brand text, -- Brand of the medication
        description text -- Description of the medication
    );

    CREATE TABLE IF NOT EXISTS trained_in (
        physician number, -- ID of the physician
        treatment number, -- ID of the treatment
        certificationdate time, -- Date of certification
        certificationexpires time -- Date of expiration of certification
    );

    CREATE TABLE IF NOT EXISTS prescribes (
        physician number, -- ID of the physician
        patient number, -- ID of the patient
        medication number, -- ID of the medication
        date time, -- Date of prescription
        appointment number, -- ID of the appointment
        dose text -- Dose of the medication
    );

    CREATE TABLE IF NOT EXISTS appointment (
        appointmentid number, -- ID of the appointment
        patient number, -- ID of the patient
        prepnurse number, -- ID of the prep nurse
        physician number, -- ID of the physician
        start time, -- Start time of the appointment
        end time, -- End time of the appointment
        examinationroom text -- Name of the examination room
    );

    CREATE TABLE IF NOT EXISTS physician (
        employeeid number, -- ID of the employee
        name text, -- Name of the physician
        position text, -- Position of the physician
        ssn number -- Social Security Number of the physician
    );

    CREATE TABLE IF NOT EXISTS stay (
        stayid number, -- ID of the stay
        patient number, -- ID of the patient
        room number, -- ID of the room
        staystart time, -- Start time of the stay
        stayend time -- End time of the stay
    );

    CREATE TABLE IF NOT EXISTS patient (
        ssn number, -- Social Security Number of the patient
        name text, -- Name of the patient
        address text, -- Address of the patient
        phone text, -- Phone number of the patient
        insuranceid number, -- ID of the insurance
        pcp number -- ID of the primary care physician
    );

    CREATE TABLE IF NOT EXISTS nurse (
        employeeid number, -- ID of the employee
        name text, -- Name of the nurse
        position text, -- Position of the nurse
        registered boolean, -- Whether the nurse is registered or not
        ssn number -- Social Security Number of the nurse
    );

    CREATE TABLE IF NOT EXISTS room (
        roomnumber number, -- Number of the room
        roomtype text, -- Type of the room
        blockfloor number, -- Floor number of the block
        blockcode number, -- Code of the block
        unavailable boolean -- Whether the room is unavailable or not
    );

    CREATE TABLE IF NOT EXISTS procedures (
        code number, -- Code of the procedure
        name text, -- Name of the procedure
        cost number -- Cost of the procedure
    );

    CREATE TABLE IF NOT EXISTS department (
        departmentid number, -- ID of the department
        name text, -- Name of the department
        head number -- ID of the head of the department
    );

    CREATE TABLE IF NOT EXISTS affiliated_with (
        physician number, -- ID of the physician
        department number, -- ID of the department
        primaryaffiliation boolean -- Whether the affiliation is primary or not
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS journal (
        Theme VARCHAR, -- Theme of the journal
        Journal\_ID VARCHAR -- ID of the journal
    );

    CREATE TABLE IF NOT EXISTS journal\_committee (
        Editor\_ID VARCHAR, -- ID of the editor
        Journal\_ID VARCHAR -- ID of the journal
    );

    CREATE TABLE IF NOT EXISTS editor (
        Name VARCHAR, -- Name of the editor
        age VARCHAR, -- Age of the editor
        Editor\_ID VARCHAR -- ID of the editor
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS inst (
        instid number, -- ID of the institution
        name text, -- Name of the institution
        country text -- Country of the institution
    );

    CREATE TABLE IF NOT EXISTS authors (
        authid number, -- ID of the author
        lname text, -- Last name of the author
        fname text -- First name of the author
    );

    CREATE TABLE IF NOT EXISTS papers (
        paperid number, -- ID of the paper
        title text -- Title of the paper
    );

    CREATE TABLE IF NOT EXISTS authorship (
        authid number, -- ID of the author
        instid number, -- ID of the institution
        paperid number, -- ID of the paper
        authorder number -- Order of authorship
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject\_id text, -- ID of the subject
        hadm\_id text, -- ID of the hospital admission
        icustay\_id text, -- ID of the ICU stay
        drug\_type text, -- Type of drug
        drug text, -- Name of the drug
        formulary\_drug\_cd text, -- Code of the drug in the formulary
        route text, -- Route of administration
        drug\_dose text -- Dose of the drug
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject\_id text, -- ID of the subject
        hadm\_id text, -- ID of the hospital admission
        name text, -- Name of the subject
        marital\_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission\_type text, -- Type of admission
        days\_stay text, -- Duration of stay
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of the subject
        expire\_flag text, -- Flag indicating if the subject is deceased
        admission\_location text, -- Location of admission
        discharge\_location text, -- Location of discharge
        diagnosis text, -- Diagnosis of the subject
        dod text, -- Date of death of the subject
        dob\_year text, -- Year of birth of the subject
        dod\_year text, -- Year of death of the subject
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );

    CREATE TABLE IF NOT EXISTS lab (
        subject\_id text, -- ID of the subject
        hadm\_id text, -- ID of the hospital admission
        itemid text, -- ID of the lab item
        charttime text, -- Time of the lab result
        flag text, -- Flag indicating the result of the lab test
        value\_unit text, -- Unit of measurement of the lab result
        label text, -- Label of the lab result
        fluid text -- Fluid used in the lab test
    );

    CREATE TABLE IF NOT EXISTS procedures (
        subject\_id text, -- ID of the subject
        hadm\_id text, -- ID of the hospital admission
        icd9\_code text, -- Code of the procedure in the ICD-9 classification system
        short\_title text, -- Short title of the procedure
        long\_title text -- Long title of the procedure
    );

    CREATE TABLE IF NOT EXISTS diagnoses (
        subject\_id text, -- ID of the subject
        hadm\_id text, -- ID of the hospital admission
        icd9\_code text, -- Code of the diagnosis in the ICD-9 classification system
        short\_title text, -- Short title of the diagnosis
        long\_title text -- Long title of the diagnosis
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS bank (
        branch\_id number, -- ID of the bank branch
        bname text, -- Name of the bank
        no\_of\_customers number, -- Number of customers at the branch
        city text, -- City of the branch
        state text -- State of the branch
    );

    CREATE TABLE IF NOT EXISTS loan (
        loan\_id text, -- ID of the loan
        loan\_type text, -- Type of the loan
        cust\_id text, -- ID of the customer
        branch\_id text, -- ID of the bank branch
        amount number -- Amount of the loan
    );

    CREATE TABLE IF NOT EXISTS customer (
        cust\_id text, -- ID of the customer
        cust\_name text, -- Name of the customer
        acc\_type text, -- Type of account
        acc\_bal number, -- Balance of the account
        no\_of\_loans number, -- Number of loans taken by the customer
        credit\_score number, -- Credit score of the customer
        branch\_id number, -- ID of the bank branch
        state text -- State of the customer
    );
    '''
]

context_lvl4 = [
    '''
    CREATE TABLE IF NOT EXISTS staff (
        staff_first_name VARCHAR, -- First name of the staff member
        staff_last_name VARCHAR, -- Last name of the staff member
        staff_id VARCHAR -- ID of the staff member
    );

    CREATE TABLE IF NOT EXISTS problems (
        product_id VARCHAR, -- ID of the product
        reported_by_staff_id VARCHAR -- ID of the staff member who reported the problem
    );

    CREATE TABLE IF NOT EXISTS product (
        product_name VARCHAR, -- Name of the product
        product_id VARCHAR -- ID of the product
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid VARCHAR, -- ID of the dorm
        amenid VARCHAR -- ID of the amenity
    );

    CREATE TABLE IF NOT EXISTS dorm (
        dorm_name VARCHAR, -- Name of the dorm
        dormid VARCHAR -- ID of the dorm
    );

    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid VARCHAR, -- ID of the amenity
        amenity_name VARCHAR -- Name of the amenity
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS department (
        dept_code VARCHAR, -- Code of the department
        dept_name VARCHAR -- Name of the department
    );

    CREATE TABLE IF NOT EXISTS CLASS (
        prof_num VARCHAR -- Number of the professor
    );

    CREATE TABLE IF NOT EXISTS professor (
        prof_high_degree VARCHAR, -- Highest degree of the professor
        emp_num VARCHAR, -- Number of the employee
        dept_code VARCHAR -- Code of the department
    );

    CREATE TABLE IF NOT EXISTS employee (
        emp_fname VARCHAR, -- First name of the employee
        emp_num VARCHAR -- Number of the employee
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS customer_orders (
        customer_id VARCHAR, -- ID of the customer
        order_id VARCHAR -- ID of the order
    );

    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR -- Name of the customer
    );

    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR, -- ID of the product
        product_details VARCHAR -- Details of the product
    );

    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR, -- ID of the order
        product_id VARCHAR -- ID of the product
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS student (
        stuid number, -- ID of the student
        lname text, -- Last name of the student
        fname text, -- First name of the student
        age number, -- Age of the student
        sex text, -- Sex of the student
        major number, -- Major of the student
        advisor number, -- ID of the advisor
        city_code text -- Code of the city
    );

    CREATE TABLE IF NOT EXISTS lives_in (
        stuid number, -- ID of the student
        dormid number, -- ID of the dorm
        room_number number -- Number of the room
    );

    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid number, -- ID of the dorm
        amenid number -- ID of the amenity
    );

    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid number, -- ID of the amenity
        amenity_name text -- Name of the amenity
    );

    CREATE TABLE IF NOT EXISTS dorm (
        dormid number, -- ID of the dorm
        dorm_name text, -- Name of the dorm
        student_capacity number, -- Capacity of the dorm
        gender text -- Gender of the dorm
    );
    '''
]

5

#### Resultados de relaciones entre tablas

In [ ]:
gpt_relations_lvl1 = [
    '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found'
    ]

gpt_relations_lvl2 = [
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- university.school_id can be joined with basketball_match.school_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- artists.id can be joined with albums.artist_id',
'-- customers.customer_id can be joined with customer_orders.customer_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- bank.branch_id can be joined with loan.branch_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id'
]

gpt_relations_lvl3 = [
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- participates_in.actid can be joined with activity.actid, -- participates_in.actid can be joined with activity.actid',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- student.stuid can be joined with lives_in.stuid, -- lives_in.dormid can be joined with dorm.dormid',
'-- campuses.id can be joined with faculty.campus, -- campuses.id can be joined with degrees.campus',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- exhibition_record.exhibition_id can be joined with exhibition.exhibition_id',
'-- physician.employeeid can be joined with prescribes.physician, -- prescribes.medication can be joined with medication.code',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- enrolled_in.stuid can be joined with student.StuID, -- gradeconversion.lettergrade can be joined with enrolled_in.Grade',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- papers.paperid can be joined with authorship.paperid, -- authorship.instid can be joined with inst.instid',
'-- customers.customer_id can be joined with customer_orders.customer_id, -- customer_orders.order_id can be joined with order_items.order_id',
'-- patient.SSN can be joined with prescribes.patient, -- prescribes.physician can be joined with physician.employeeid'
]

gpt_relations_lvl4 = [
'-- salary.player_id can be joined with player.player_id, -- salary.team_id can be joined with team.team_id_br',
'-- performance.bandmate can be joined with band.id, -- songs.songid can be joined with performance.songid, -- songs.songid can be joined with instruments.songid, -- band.id can be joined with instruments.bandmateid',
'-- employee.eid can be joined with certificate.eid, -- aircraft.aid can be joined with certificate.aid',
'-- tasks.project_id can be joined with projects.project_id, -- tasks.project_id can be joined with project_outcomes.project_id',
'-- customers.customer_id can be joined with orders.customer_id, -- orders.order_id can be joined with order_items.order_id, -- order_items.product_id can be joined with products.product_id',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlist_tracks.playlist_id can be joined with playlists.id',
'-- swimmer.id can be joined with record.swimmer_id, -- record.event_id can be joined with event.id, -- event.stadium_id can be joined with stadium.id',
'-- Tourist_Attractions.Tourist_Attraction_ID can be joined with VISITS.Tourist_Attraction_ID, -- VISITORS.Tourist_ID can be joined with VISITS.Tourist_ID',
'-- problems.product_id can be joined with product.product_id, -- problems.reported_by_staff_id can be joined with staff.staff_id',
'-- tracks.id can be joined with invoice_lines.track_id, -- invoices.id can be joined with invoice_lines.invoice_id, -- invoices.customer_id can be joined with customers.id',
'-- employee.emp_num can be joined with professor.emp_num, -- professor.dept_code can be joined with department.dept_code',
'-- class.class_code can be joined with enroll.class_code, -- class.crs_code can be joined with course.crs_code, -- course.dept_code can be joined with department.dept_code',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlist_tracks.playlist_id can be joined with playlists.id',
'-- personfriend.name can be joined with person.name, -- personfriend.friend can be joined with personfriend.name, -- personfriend.friend can be joined with personfriend.name',
'-- affected_region.region_id can be joined with region.region_id, -- affected_region.storm_id can be joined with storm.storm_id',
'-- Album.AlbumId can be joined with Track.AlbumId, -- Track.GenreID can be joined with Genre.GenreID',
'-- assignedto.project can be joined with projects.code, -- assignedto.scientist can be joined with scientists.ssn',
'-- student.stuid can be joined with lives_in.stuid, -- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid',
'-- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid',
'-- races.raceid can be joined with results.raceid, -- results.driverid can be joined with drivers.driverid'
]
print(len(gpt_relations_lvl1), len(gpt_relations_lvl2), len(gpt_relations_lvl3), len(gpt_relations_lvl4))


#### Resultados de comentarios en las tablas

In [ ]:
gpt_context_lvl1 = [
    '''
    CREATE TABLE IF NOT EXISTS table_19662262_6 (
        bbm VARCHAR, -- BBM (Blackberry Messenger) ID
        strike_rate VARCHAR -- Strike rate in cricket
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_26 (
        max_1_min_wind_mph__km_h_ VARCHAR, -- Maximum wind speed in mph or km/h
        dates_active VARCHAR -- Dates when the record is active
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_11960944_11 (
        location_attendance VARCHAR, -- Location of attendance
        high_assists VARCHAR -- High assists in a game
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of charting
        flag text, -- Indicator flag
        value_unit text, -- Unit of value
        label text, -- Label information
        fluid text -- Type of fluid
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of diagnosis
        long_title text -- Long title of diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of drug
        drug text, -- Drug name
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Drug dosage
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of procedure
        long_title text -- Long title of procedure
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        name text, -- Name of patient
        marital_status text, -- Marital status of patient
        age text, -- Age of patient
        dob text, -- Date of birth of patient
        gender text, -- Gender of patient
        language text, -- Language spoken by patient
        religion text, -- Religion of patient
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of patient
        expire_flag text, -- Flag indicating expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Admission time
        dischtime text, -- Discharge time
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_16225511_2 (
        school VARCHAR, -- School name
        percent VARCHAR -- Percentage
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_96 (
        school VARCHAR, -- School name
        player VARCHAR -- Player name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_27225944_3 (
        outgoing_manager VARCHAR, -- Name of outgoing manager
        manner_of_departure VARCHAR, -- Manner of departure
        position_in_table VARCHAR -- Position in the league table
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_24706337_1 (
        victor VARCHAR, -- Victor's name
        name_of_the_war VARCHAR -- Name of the war
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_16295365_1 (
        conference VARCHAR, -- Conference name
        school VARCHAR -- School name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_2051288_1 (
        _2000_ VARCHAR, -- Value for the year 2000
        population__may INTEGER, -- Population in May
        location VARCHAR, -- Location
        barangay VARCHAR -- Barangay name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_57 (
        rank INTEGER, -- Rank
        rider VARCHAR -- Rider name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of charting
        flag text, -- Indicator flag
        value_unit text, -- Unit of value
        label text, -- Label information
        fluid text -- Type of fluid
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of diagnosis
        long_title text -- Long title of diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        name text, -- Name of patient
        marital_status text, -- Marital status of patient
        age text, -- Age of patient
        dob text, -- Date of birth of patient
        gender text, -- Gender of patient
        language text, -- Language spoken by patient
        religion text, -- Religion of patient
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of patient
        expire_flag text, -- Flag indicating expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Admission time
        dischtime text, -- Discharge time
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of procedure
        long_title text -- Long title of procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of drug
        drug text, -- Drug name
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Drug dosage
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_17 (
        wins VARCHAR, -- Number of wins
        points INTEGER -- Points
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_89 (
        one_way VARCHAR, -- Direction
        miles_ INTEGER, -- Miles traveled
        fans_took VARCHAR -- Number of fans took
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_13770460_3 (
        eu VARCHAR, -- European Union abbreviation
        country VARCHAR -- Country name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS party (
        party_id number, -- Party ID
        party_theme text, -- Party theme
        location text, -- Location of party
        first_year text, -- First year of party
        last_year text, -- Last year of party
        number_of_hosts number -- Number of hosts
    );
    CREATE TABLE IF NOT EXISTS host (
        host_id number, -- Host ID
        name text, -- Name of host
        nationality text, -- Nationality of host
        age text -- Age of host
    );
    CREATE TABLE IF NOT EXISTS party_host (
        party_id number, -- Party ID
        host_id number, -- Host ID
        is_main_in_charge others -- Indicator for main host
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS participants (
        participant_type_code VARCHAR -- Participant type code
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_80 (
        score VARCHAR, -- Score
        year VARCHAR -- Year
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_99 (
        spine VARCHAR, -- Spine
        number_of_issues VARCHAR -- Number of issues
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS trip (
        id VARCHAR, -- Trip ID
        duration VARCHAR -- Duration of trip
    );
    '''
]


In [ ]:
gpt_context_lvl2 = [
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    '''
]
gpt_context_lvl2 += [
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    '''
]
gpt_context_lvl2 += [
    '''
    CREATE TABLE IF NOT EXISTS basketball_match (
        team_name VARCHAR, -- Name of the basketball team
        school_id VARCHAR -- Identifier for the school
    );
    CREATE TABLE IF NOT EXISTS university (
        school_id VARCHAR -- Identifier for the school
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    '''
]
gpt_context_lvl2 += [
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    '''
]
gpt_context_lvl2 += [
    '''
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        playlist_id number, -- Identifier for the playlist
        track_id number -- Identifier for the track
    );
    CREATE TABLE IF NOT EXISTS customers (
        id number, -- Identifier for the customer
        first_name text, -- First name of the customer
        last_name text, -- Last name of the customer
        company text, -- Company of the customer
        address text, -- Address of the customer
        city text, -- City of the customer
        state text, -- State of the customer
        country text, -- Country of the customer
        postal_code text, -- Postal code of the customer
        phone text, -- Phone number of the customer
        fax text, -- Fax number of the customer
        email text, -- Email address of the customer
        support_rep_id number -- Identifier for the support representative
    );
    CREATE TABLE IF NOT EXISTS albums (
        id number, -- Identifier for the album
        title text, -- Title of the album
        artist_id number -- Identifier for the artist
    );
    CREATE TABLE IF NOT EXISTS tracks (
        id number, -- Identifier for the track
        name text, -- Name of the track
        album_id number, -- Identifier for the album
        media_type_id number, -- Identifier for the media type
        genre_id number, -- Identifier for the genre
        composer text, -- Composer of the track
        milliseconds number, -- Length of the track in milliseconds
        bytes number, -- Size of the track in bytes
        unit_price number -- Unit price of the track
    );
    CREATE TABLE IF NOT EXISTS invoices (
        id number, -- Identifier for the invoice
        customer_id number, -- Identifier for the customer
        invoice_date time, -- Date of the invoice
        billing_address text, -- Billing address for the invoice
        billing_city text, -- Billing city for the invoice
        billing_state text, -- Billing state for the invoice
        billing_country text, -- Billing country for the invoice
        billing_postal_code text, -- Billing postal code for the invoice
        total number -- Total amount for the invoice
    );
    CREATE TABLE IF NOT EXISTS artists (
        id number, -- Identifier for the artist
        name text -- Name of the artist
    );
    CREATE TABLE IF NOT EXISTS employees (
        id number, -- Identifier for the employee
        last_name text, -- Last name of the employee
        first_name text, -- First name of the employee
        title text, -- Title of the employee
        reports_to number, -- Identifier for the employee's supervisor
        birth_date time, -- Birth date of the employee
        hire_date time, -- Hire date of the employee
        address text, -- Address of the employee
        city text, -- City of the employee
        state text, -- State of the employee
        country text, -- Country of the employee
        postal_code text, -- Postal code of the employee
        phone text, -- Phone number of the employee
        fax text, -- Fax number of the employee
        email text -- Email address of the employee
    );
    CREATE TABLE IF NOT EXISTS sqlite_sequence (
        name text, -- Name of the sequence
        seq text -- Current value of the sequence
    );
    CREATE TABLE IF NOT EXISTS playlists (
        id number, -- Identifier for the playlist
        name text -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS genres (
        id number, -- Identifier for the genre
        name text -- Name of the genre
    );
    CREATE TABLE IF NOT EXISTS media_types (
        id number, -- Identifier for the media type
        name text -- Name of the media type
    );
    CREATE TABLE IF NOT EXISTS invoice_lines (
        id number, -- Identifier for the invoice line
        invoice_id number, -- Identifier for the invoice
        track_id number, -- Identifier for the track
        unit_price number, -- Unit price of the track
        quantity number -- Quantity of the track
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS suppliers (
        supplier_id number, -- Identifier for the supplier
        supplier_name text, -- Name of the supplier
        supplier_phone text -- Phone number of the supplier
    );
    CREATE TABLE IF NOT EXISTS addresses (
        address_id number, -- Identifier for the address
        address_details text -- Details of the address
    );
    CREATE TABLE IF NOT EXISTS customer_addresses (
        customer_id number, -- Identifier for the customer
        address_id number, -- Identifier for the address
        date_from time, -- Start date for the address
        date_to time -- End date for the address
    );
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id number, -- Identifier for the order item
        order_id number, -- Identifier for the order
        product_id number -- Identifier for the product
    );
    CREATE TABLE IF NOT EXISTS department_store_chain (
        dept_store_chain_id number, -- Identifier for the department store chain
        dept_store_chain_name text -- Name of the department store chain
    );
    CREATE TABLE IF NOT EXISTS department_stores (
        dept_store_id number, -- Identifier for the department store
        dept_store_chain_id number, -- Identifier for the department store chain
        store_name text, -- Name of the department store
        store_address text, -- Address of the department store
        store_phone text, -- Phone number of the department store
        store_email text -- Email address of the department store
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_id number, -- Identifier for the customer
        payment_method_code text, -- Code for the payment method
        customer_code text, -- Code for the customer
        customer_name text, -- Name of the customer
        customer_address text, -- Address of the customer
        customer_phone text, -- Phone number of the customer
        customer_email text -- Email address of the customer
    );
    CREATE TABLE IF NOT EXISTS supplier_addresses (
        supplier_id number, -- Identifier for the supplier
        address_id number, -- Identifier for the address
        date_from time, -- Start date for the address
        date_to time -- End date for the address
    );
    CREATE TABLE IF NOT EXISTS staff_department_assignments (
        staff_id number, -- Identifier for the staff member
        department_id number, -- Identifier for the department
        date_assigned_from time, -- Start date for the assignment
        job_title_code text, -- Code for the job title
        date_assigned_to time -- End date for the assignment
    );
    CREATE TABLE IF NOT EXISTS staff (
        staff_id number, -- Identifier for the staff member
        staff_gender text, -- Gender of the staff member
        staff_name text -- Name of the staff member
    );
    CREATE TABLE IF NOT EXISTS product_suppliers (
        product_id number, -- Identifier for the product
        supplier_id number, -- Identifier for the supplier
        date_supplied_from time, -- Start date for the supply
        date_supplied_to time, -- End date for the supply
        total_amount_purchased text, -- Total amount purchased of the product
        total_value_purchased number -- Total value purchased of the product
    );
    CREATE TABLE IF NOT EXISTS customer_orders (
        order_id number, -- Identifier for the order
        customer_id number, -- Identifier for the customer
        order_status_code text, -- Code for the order status
        order_date time -- Date of the order
    );
    CREATE TABLE IF NOT EXISTS departments (
        department_id number, -- Identifier for the department
        dept_store_id number, -- Identifier for the department store
        department_name text -- Name of the department
    );
    CREATE TABLE IF NOT EXISTS products (
        product_id number, -- Identifier for the product
        product_type_code text, -- Code for the product type
        product_name text, -- Name of the product
        product_price number -- Price of the product
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    '''
]
gpt_context_lvl2 += [
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );

    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );

    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );

    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS bank (
        bname VARCHAR, -- Name of the bank
        branch_id VARCHAR -- Identifier for a bank branch
    );

    CREATE TABLE IF NOT EXISTS loan (
        branch_id VARCHAR -- Identifier for a bank branch
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    '''
]


In [ ]:
gpt_context_lvl3 = [
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS activity (
        actid number, -- Unique identifier for an activity
        activity_name text -- Name of the activity
    );
    CREATE TABLE IF NOT EXISTS faculty_participates_in (
        facid number, -- Unique identifier for a faculty member
        actid number -- Unique identifier for an activity
    );
    CREATE TABLE IF NOT EXISTS faculty (
        facid number, -- Unique identifier for a faculty member
        lname text, -- Last name of the faculty member
        fname text, -- First name of the faculty member
        rank text, -- Rank of the faculty member (e.g. professor, associate professor)
        sex text, -- Gender of the faculty member
        phone number, -- Phone number of the faculty member
        room text, -- Room number of the faculty member's office
        building text -- Building where the faculty member's office is located
    );
    CREATE TABLE IF NOT EXISTS participates_in (
        stuid number, -- Unique identifier for a student
        actid number -- Unique identifier for an activity
    );
    CREATE TABLE IF NOT EXISTS student (
        stuid number, -- Unique identifier for a student
        lname text, -- Last name of the student
        fname text, -- First name of the student
        age number, -- Age of the student
        sex text, -- Gender of the student
        major number, -- Unique identifier for the student's major
        advisor number, -- Unique identifier for the student's advisor
        city_code text -- Code for the city where the student lives
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid number, -- Unique identifier for a dorm
        amenid number -- Unique identifier for an amenity
    );
    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid number, -- Unique identifier for an amenity
        amenity_name text -- Name of the amenity
    );
    CREATE TABLE IF NOT EXISTS lives_in (
        stuid number, -- Unique identifier for a student
        dormid number, -- Unique identifier for a dorm
        room_number number -- Room number where the student lives
    );
    CREATE TABLE IF NOT EXISTS student (
        stuid number, -- Unique identifier for a student
        lname text, -- Last name of the student
        fname text, -- First name of the student
        age number, -- Age of the student
        sex text, -- Gender of the student
        major number, -- Unique identifier for the student's major
        advisor number, -- Unique identifier for the student's advisor
        city_code text -- Code for the city where the student lives
    );
    CREATE TABLE IF NOT EXISTS dorm (
        dormid number, -- Unique identifier for a dorm
        dorm_name text, -- Name of the dorm
        student_capacity number, -- Maximum number of students that can live in the dorm
        gender text -- Gender of the students that live in the dorm (e.g. male, female, coed)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS discipline_enrollments (
        campus number, -- Unique identifier for a campus
        discipline number, -- Unique identifier for a discipline
        year number, -- Year of enrollment
        undergraduate number, -- Number of undergraduate students enrolled in the discipline
        graduate number -- Number of graduate students enrolled in the discipline
    );
    CREATE TABLE IF NOT EXISTS csu_fees (
        campus number, -- Unique identifier for a campus
        year number, -- Year of enrollment
        campusfee number -- Fee for the campus
    );
    CREATE TABLE IF NOT EXISTS enrollments (
        campus number, -- Unique identifier for a campus
        year number, -- Year of enrollment
        totalenrollment_ay number, -- Total number of students enrolled in the academic year
        fte_ay number -- Full-time equivalent enrollment in the academic year
    );
    CREATE TABLE IF NOT EXISTS degrees (
        year number, -- Year of graduation
        campus number, -- Unique identifier for a campus
        degrees number -- Number of degrees awarded
    );
    CREATE TABLE IF NOT EXISTS faculty (
        campus number, -- Unique identifier for a campus
        year number, -- Year of employment
        faculty number -- Number of faculty members employed
    );
    CREATE TABLE IF NOT EXISTS campuses (
        id number, -- Unique identifier for a campus
        campus text, -- Name of the campus
        location text, -- Location of the campus
        county text, -- County where the campus is located
        year number -- Year the campus was established
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of the chart
        flag text, -- Flag for the lab result
        value_unit text, -- Unit of the lab value
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 procedure code
        short_title text, -- Short title of the procedure
        long_title text -- Long title of the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 diagnosis code
        short_title text, -- Short title of the diagnosis
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of the drug
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Duration of stay in the hospital
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis information
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of the chart
        flag text, -- Flag for the lab result
        value_unit text, -- Unit of the lab value
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 diagnosis code
        short_title text, -- Short title of the diagnosis
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of the drug
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Duration of stay in the hospital
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis information
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 procedure code
        short_title text, -- Short title of the procedure
        long_title text -- Long title of the procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS artist (
        artist_id number, -- Unique identifier for the artist
        name text, -- Name of the artist
        country text, -- Country of origin of the artist
        year_join number, -- Year the artist joined the exhibition
        age number -- Age of the artist
    );
    CREATE TABLE IF NOT EXISTS exhibition (
        exhibition_id number, -- Unique identifier for the exhibition
        year number, -- Year of the exhibition
        theme text, -- Theme of the exhibition
        artist_id number, -- Unique identifier for the artist participating in the exhibition
        ticket_price number -- Price of admission for the exhibition
    );
    CREATE TABLE IF NOT EXISTS exhibition_record (
        exhibition_id number, -- Unique identifier for the exhibition
        date text, -- Date of the exhibition record
        attendance number -- Number of attendees at the exhibition on the given date
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS on_call (
        nurse number, -- Unique identifier for the nurse on call
        blockfloor number, -- Floor of the block where the nurse is on call
        blockcode number, -- Code for the block where the nurse is on call
        oncallstart time, -- Start time of the nurse's on-call shift
        oncallend time -- End time of the nurse's on-call shift
    );
    CREATE TABLE IF NOT EXISTS nurse (
        employeeid number, -- Unique identifier for the nurse
        name text, -- Name of the nurse
        position text, -- Position of the nurse
        registered boolean, -- Flag indicating if the nurse is registered
        ssn number -- Social security number of the nurse
    );
    CREATE TABLE IF NOT EXISTS department (
        departmentid number, -- Unique identifier for the department
        name text, -- Name of the department
        head number -- Unique identifier for the head of the department
    );
    CREATE TABLE IF NOT EXISTS block (
        blockfloor number, -- Floor of the block
        blockcode number -- Code for the block
    );
    CREATE TABLE IF NOT EXISTS medication (
        code number, -- Unique identifier for the medication
        name text, -- Name of the medication
        brand text, -- Brand name of the medication
        description text -- Description of the medication
    );
    CREATE TABLE IF NOT EXISTS trained_in (
        physician number, -- Unique identifier for the physician
        treatment number, -- Unique identifier for the treatment the physician is trained in
        certificationdate time, -- Date the physician received certification for the treatment
        certificationexpires time -- Date the physician's certification for the treatment expires
    );
    CREATE TABLE IF NOT EXISTS prescribes (
        physician number, -- Unique identifier for the physician
        patient number, -- Unique identifier for the patient
        medication number, -- Unique identifier for the medication prescribed
        date time, -- Date the medication was prescribed
        appointment number, -- Unique identifier for the appointment associated with the prescription
        dose text -- Dose of the medication prescribed
    );
    CREATE TABLE IF NOT EXISTS procedures (
        code number, -- Unique identifier for the procedure
        name text, -- Name of the procedure
        cost number -- Cost of the procedure
    );
    CREATE TABLE IF NOT EXISTS stay (
        stayid number, -- Unique identifier for the stay
        patient number, -- Unique identifier for the patient
        room number, -- Room number for the stay
        staystart time, -- Start time of the stay
        stayend time -- End time of the stay
    );
    CREATE TABLE IF NOT EXISTS patient (
        ssn number, -- Social security number of the patient
        name text, -- Name of the patient
        address text, -- Address of the patient
        phone text, -- Phone number of the patient
        insuranceid number, -- Unique identifier for the patient's insurance
        pcp number -- Unique identifier for the patient's primary care physician
    );
    CREATE TABLE IF NOT EXISTS undergoes (
        patient number, -- Unique identifier for the patient
        procedures number, -- Unique identifier for the procedure the patient undergoes
        stay number, -- Unique identifier for the stay associated with the procedure
        dateundergoes time, -- Date the procedure was performed
        physician number, -- Unique identifier for the physician who performed the procedure
        assistingnurse number -- Unique identifier for the nurse who assisted with the procedure
    );
    CREATE TABLE IF NOT EXISTS appointment (
        appointmentid number, -- Unique identifier for the appointment
        patient number, -- Unique identifier for the patient associated with the appointment
        prepnurse number, -- Unique identifier for the nurse who prepped the patient for the appointment
        physician number, -- Unique identifier for the physician associated with the appointment
        start time, -- Start time of the appointment
        end time, -- End time of the appointment
        examinationroom text -- Room where the appointment took place
    );
    CREATE TABLE IF NOT EXISTS affiliated_with (
        physician number, -- Unique identifier for the physician
        department number, -- Unique identifier for the department the physician is affiliated with
        primaryaffiliation boolean -- Flag indicating if the affiliation is the physician's primary affiliation
    );
    CREATE TABLE IF NOT EXISTS physician (
        employeeid number, -- Unique identifier for the physician
        name text, -- Name of the physician
        position text, -- Position of the physician
        ssn number -- Social security number of the physician
    );
    CREATE TABLE IF NOT EXISTS room (
        roomnumber number, -- Room number
        roomtype text, -- Type of room
        blockfloor number, -- Floor of the block where the room is located
        blockcode number, -- Code for the block where the room is located
        unavailable boolean -- Flag indicating if the room is unavailable
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icd9_code text -- ICD-9 code for the diagnosis,
        short_title text -- Short title of the diagnosis,
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icd9_code text -- ICD-9 code for the procedure,
        short_title text -- Short title of the procedure,
        long_title text -- Long title of the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icustay_id text -- Intensive care stay identifier,
        drug_type text -- Type of the drug,
        drug text -- Name of the drug,
        formulary_drug_cd text -- Formulary drug code,
        route text -- Route of administration,
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        name text -- Name of the subject,
        marital_status text -- Marital status of the subject,
        age text -- Age of the subject,
        dob text -- Date of birth of the subject,
        gender text -- Gender of the subject,
        language text -- Language spoken by the subject,
        religion text -- Religion of the subject,
        admission_type text -- Type of admission,
        days_stay text -- Duration of stay in the hospital,
        insurance text -- Insurance information,
        ethnicity text -- Ethnicity of the subject,
        expire_flag text -- Flag indicating if the subject is deceased,
        admission_location text -- Location of admission,
        discharge_location text -- Location of discharge,
        diagnosis text -- Diagnosis information,
        dod text -- Date of death,
        dob_year text -- Year of birth,
        dod_year text -- Year of death,
        admittime text -- Time of admission,
        dischtime text -- Time of discharge,
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        itemid text -- Identifier for the lab item,
        charttime text -- Time of the lab test,
        flag text -- Flag for the lab test,
        value_unit text -- Unit of measurement for the lab value,
        label text -- Label for the lab test,
        fluid text -- Type of fluid tested
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS STUDENT (
        Fname VARCHAR -- First name of the student,
        StuID VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS ENROLLED_IN (
        Grade VARCHAR -- Grade of the student,
        StuID VARCHAR -- Unique identifier for the student
    );
    CREATE TABLE IF NOT EXISTS GRADECONVERSION (
        gradepoint VARCHAR -- Grade point,
        lettergrade VARCHAR -- Letter grade
    );
    ''',
    '''
    -- (Same as DDL 1)
    ''',
    '''
    -- (Same as DDL 1, but with tables in a different order)
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        name text -- Name of the subject,
        marital_status text -- Marital status of the subject,
        age text -- Age of the subject,
        dob text -- Date of birth of the subject,
        gender text -- Gender of the subject,
        language text -- Language spoken by the subject,
        religion text -- Religion of the subject,
        admission_type text -- Type of admission,
        days_stay text -- Duration of stay in the hospital,
        insurance text -- Insurance information,
        ethnicity text -- Ethnicity of the subject,
        expire_flag text -- Flag indicating if the subject is deceased,
        admission_location text -- Location of admission,
        discharge_location text -- Location of discharge,
        diagnosis text -- Diagnosis information,
        dod text -- Date of death,
        dob_year text -- Year of birth,
        dod_year text -- Year of death,
        admittime text -- Time of admission,
        dischtime text -- Time of discharge,
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icd9_code text -- ICD-9 code for the diagnosis,
        short_title text -- Short title of the diagnosis,
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icd9_code text -- ICD-9 code for the procedure,
        short_title text -- Short title of the procedure,
        long_title text -- Long title of the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        itemid text -- Identifier for the lab item,
        charttime text -- Time of the lab test,
        flag text -- Flag for the lab test,
        value_unit text -- Unit of measurement for the lab value,
        label text -- Label for the lab test,
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text -- Unique identifier for the subject,
        hadm_id text -- Hospital admission identifier,
        icustay_id text -- Intensive care stay identifier,
        drug_type text -- Type of the drug,
        drug text -- Name of the drug,
        formulary_drug_cd text -- Formulary drug code,
        route text -- Route of administration,
        drug_dose text -- Dose of the drug
    );
    '''
]
gpt_context_lvl3 += [
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of the chart
        flag text, -- Flag for the lab result
        value_unit text, -- Unit of the lab value
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Duration of stay in the hospital
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis information
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of the drug
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 diagnosis code
        short_title text, -- Short title of the diagnosis
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 procedure code
        short_title text, -- Short title of the procedure
        long_title text -- Long title of the procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        name text, -- Name of the subject
        marital_status text, -- Marital status of the subject
        age text, -- Age of the subject
        dob text, -- Date of birth of the subject
        gender text, -- Gender of the subject
        language text, -- Language spoken by the subject
        religion text, -- Religion of the subject
        admission_type text, -- Type of admission
        days_stay text, -- Duration of stay in the hospital
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of the subject
        expire_flag text, -- Flag indicating if the subject is deceased
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis information
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 diagnosis code
        short_title text, -- Short title of the diagnosis
        long_title text -- Long title of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 procedure code
        short_title text, -- Short title of the procedure
        long_title text -- Long title of the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of the drug
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for the subject
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of the chart
        flag text, -- Flag for the lab result
        value_unit text, -- Unit of the lab value
        label text, -- Label for the lab result
        fluid text -- Type of fluid used in the lab test
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS authors (
        authid number, -- Unique identifier for the author
        lname text, -- Last name of the author
        fname text -- First name of the author
    );
    CREATE TABLE IF NOT EXISTS inst (
        instid number, -- Unique identifier for the institution
        name text, -- Name of the institution
        country text -- Country of the institution
    );
    CREATE TABLE IF NOT EXISTS authorship (
        authid number, -- Unique identifier for the author
        instid number, -- Unique identifier for the institution
        paperid number, -- Unique identifier for the paper
        authorder number -- Order of the authors for the paper
    );
    CREATE TABLE IF NOT EXISTS papers (
        paperid number, -- Unique identifier for the paper
        title text -- Title of the paper
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS order_items (
        order_id number, -- Unique identifier for the order
        product_id number, -- Unique identifier for the product
        order_quantity text -- Quantity of the product in the order
    );
    CREATE TABLE IF NOT EXISTS customer_orders (
        order_id number, -- Unique identifier for the order
        customer_id number, -- Unique identifier for the customer
        order_status text, -- Status of the order
        order_date time, -- Date of the order
        order_details text -- Details of the order
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_id number, -- Unique identifier for the customer
        payment_method text, -- Payment method of the customer
        customer_name text, -- Name of the customer
        date_became_customer time, -- Date the customer joined
        other_customer_details text -- Other details about the customer
    );
    CREATE TABLE IF NOT EXISTS products (
        product_id number, -- Unique identifier for the product
        product_details text -- Details of the product
    );
    CREATE TABLE IF NOT EXISTS addresses (
        address_id number, -- Unique identifier for the address
        address_content text, -- Content of the address
        city text, -- City of the address
        zip_postcode text, -- Zip or postal code of the address
        state_province_county text, -- State, province, or county of the address
        country text, -- Country of the address
        other_address_details text -- Other details about the address
    );
    CREATE TABLE IF NOT EXISTS customer_contact_channels (
        customer_id number, -- Unique identifier for the customer
        channel_code text, -- Code for the contact channel
        active_from_date time, -- Date the channel became active
        active_to_date time, -- Date the channel became inactive
        contact_number text -- Contact number for the channel
    );
    CREATE TABLE IF NOT EXISTS customer_addresses (
        customer_id number, -- Unique identifier for the customer
        address_id number, -- Unique identifier for the address
        date_address_from time, -- Date the address became active
        address_type text, -- Type of the address
        date_address_to time -- Date the address became inactive
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS physician (
        employeeid VARCHAR, -- Unique identifier for the physician
        name VARCHAR -- Name of the physician
    );
    CREATE TABLE IF NOT EXISTS patient (
        SSN VARCHAR -- Social Security Number of the patient
    );
    CREATE TABLE IF NOT EXISTS prescribes (
        patient VARCHAR, -- Social Security Number of the patient
        physician VARCHAR -- Unique identifier for the physician
    );
    '''
]


In [ ]:
gpt_context_lvl4 = [
    '''
    CREATE TABLE IF NOT EXISTS salary (
        player_id VARCHAR, -- Unique identifier for the player
        team_id VARCHAR, -- Unique identifier for the team
        year VARCHAR -- Year of the salary record
    );
    CREATE TABLE IF NOT EXISTS player (
        name_first VARCHAR, -- First name of the player
        name_last VARCHAR, -- Last name of the player
        player_id VARCHAR -- Unique identifier for the player
    );
    CREATE TABLE IF NOT EXISTS team (
        team_id_br VARCHAR, -- Unique identifier for the team
        name VARCHAR -- Name of the team
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS tracklists (
        albumid number, -- Unique identifier for the album
        position number, -- Position of the song in the album
        songid number -- Unique identifier for the song
    );
    CREATE TABLE IF NOT EXISTS vocals (
        songid number, -- Unique identifier for the song
        bandmate number, -- Unique identifier for the bandmate
        type text -- Type of vocals (e.g., lead, backup)
    );
    CREATE TABLE IF NOT EXISTS band (
        id number, -- Unique identifier for the bandmate
        firstname text, -- First name of the bandmate
        lastname text -- Last name of the bandmate
    );
    CREATE TABLE IF NOT EXISTS songs (
        songid number, -- Unique identifier for the song
        title text -- Title of the song
    );
    CREATE TABLE IF NOT EXISTS albums (
        aid number, -- Unique identifier for the album
        title text, -- Title of the album
        year number, -- Year the album was released
        label text, -- Record label of the album
        type text -- Type of album (e.g., studio, live)
    );
    CREATE TABLE IF NOT EXISTS instruments (
        songid number, -- Unique identifier for the song
        bandmateid number, -- Unique identifier for the bandmate
        instrument text -- Instrument played by the bandmate in the song
    );
    CREATE TABLE IF NOT EXISTS performance (
        songid number, -- Unique identifier for the song
        bandmate number, -- Unique identifier for the bandmate
        stageposition text -- Position of the bandmate on stage during the performance
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS flight (
        flno number, -- Unique identifier for the flight
        origin text, -- Origin airport of the flight
        destination text, -- Destination airport of the flight
        distance number, -- Distance of the flight
        departure_date time, -- Departure date and time of the flight
        arrival_date time, -- Arrival date and time of the flight
        price number, -- Price of the flight
        aid number -- Unique identifier for the aircraft
    );
    CREATE TABLE IF NOT EXISTS aircraft (
        aid number, -- Unique identifier for the aircraft
        name text, -- Name of the aircraft
        distance number -- Distance the aircraft can travel
    );
    CREATE TABLE IF NOT EXISTS certificate (
        eid number, -- Unique identifier for the employee
        aid number -- Unique identifier for the aircraft
    );
    CREATE TABLE IF NOT EXISTS employee (
        eid number, -- Unique identifier for the employee
        name text, -- Name of the employee
        salary number -- Salary of the employee
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS grants (
        grant_id number, -- Unique identifier for the grant
        organisation_id number, -- Unique identifier for the organisation
        grant_amount number, -- Amount of money awarded for the grant
        grant_start_date time, -- Start date of the grant
        grant_end_date time, -- End date of the grant
        other_details text -- Other details about the grant
    );
    CREATE TABLE IF NOT EXISTS projects (
        project_id number, -- Unique identifier for the project
        organisation_id number, -- Unique identifier for the organisation
        project_details text -- Details about the project
    );
    CREATE TABLE IF NOT EXISTS organisation_types (
        organisation_type text, -- Type of organisation
        organisation_type_description text -- Description of the organisation type
    );
    CREATE TABLE IF NOT EXISTS tasks (
        task_id number, -- Unique identifier for the task
        project_id number, -- Unique identifier for the project
        task_details text, -- Details about the task
        eg_agree_objectives text -- Example details about agreeing on objectives
    );
    CREATE TABLE IF NOT EXISTS organisations (
        organisation_id number, -- Unique identifier for the organisation
        organisation_type text, -- Type of organisation
        organisation_details text -- Details about the organisation
    );
    CREATE TABLE IF NOT EXISTS project_staff (
        staff_id number, -- Unique identifier for the staff member
        project_id number, -- Unique identifier for the project
        role_code text, -- Code for the staff member's role
        date_from time, -- Start date of the staff member's involvement in the project
        date_to time, -- End date of the staff member's involvement in the project
        other_details text -- Other details about the staff member's involvement in the project
    );
    CREATE TABLE IF NOT EXISTS research_staff (
        staff_id number, -- Unique identifier for the staff member
        employer_organisation_id number, -- Unique identifier for the staff member's employer organisation
        staff_details text -- Details about the staff member
    );
    CREATE TABLE IF NOT EXISTS staff_roles (
        role_code text, -- Code for the staff member's role
        role_description text -- Description of the staff member's role
    );
    CREATE TABLE IF NOT EXISTS documents (
        document_id number, -- Unique identifier for the document
        document_type_code text, -- Code for the type of document
        grant_id number, -- Unique identifier for the grant associated with the document
        sent_date time, -- Date the document was sent
        response_received_date time, -- Date the response to the document was received
        other_details text -- Other details about the document
    );
    CREATE TABLE IF NOT EXISTS project_outcomes (
        project_id number, -- Unique identifier for the project
        outcome_code text, -- Code for the outcome of the project
        outcome_details text -- Details about the outcome of the project
    );
    CREATE TABLE IF NOT EXISTS document_types (
        document_type_code text, -- Code for the type of document
        document_description text -- Description of the type of document
    );
    CREATE TABLE IF NOT EXISTS research_outcomes (
        outcome_code text, -- Code for the outcome of the research
        outcome_description text -- Description of the outcome of the research
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR, -- Unique identifier for the product
        product_name VARCHAR -- Name of the product
    );
    CREATE TABLE IF NOT EXISTS order_items (
        product_id VARCHAR, -- Unique identifier for the product
        order_item_status VARCHAR, -- Status of the order item
        order_id VARCHAR -- Unique identifier for the order
    );
    CREATE TABLE IF NOT EXISTS orders (
        customer_id VARCHAR, -- Unique identifier for the customer
        order_id VARCHAR -- Unique identifier for the order
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_name VARCHAR, -- Name of the customer
        customer_id VARCHAR -- Unique identifier for the customer
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS playlists (
        id VARCHAR, -- Unique identifier for the playlist
        name VARCHAR -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        id VARCHAR -- Unique identifier for the track
    );
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        track_id VARCHAR, -- Unique identifier for the track
        playlist_id VARCHAR -- Unique identifier for the playlist
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS swimmer (
        id VARCHAR, -- Unique identifier for the swimmer
        nationality VARCHAR -- Nationality of the swimmer
    );
    CREATE TABLE IF NOT EXISTS record (
        swimmer_id VARCHAR, -- Unique identifier for the swimmer
        event_id VARCHAR -- Unique identifier for the event
    );
    CREATE TABLE IF NOT EXISTS event (
        id VARCHAR, -- Unique identifier for the event
        stadium_id VARCHAR -- Unique identifier for the stadium
    );
    CREATE TABLE IF NOT EXISTS stadium (
        name VARCHAR, -- Name of the stadium
        id VARCHAR -- Unique identifier for the stadium
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS VISITS (
        Tourist_Attraction_ID VARCHAR, -- Unique identifier for the tourist attraction
        Tourist_ID VARCHAR -- Unique identifier for the tourist
    );
    CREATE TABLE IF NOT EXISTS VISITORS (
        Tourist_Details VARCHAR, -- Details about the tourist
        Tourist_ID VARCHAR -- Unique identifier for the tourist
    );
    CREATE TABLE IF NOT EXISTS Tourist_Attractions (
        Name VARCHAR, -- Name of the tourist attraction
        Tourist_Attraction_ID VARCHAR -- Unique identifier for the tourist attraction
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS staff (
        staff_id number, -- Unique identifier for the staff member
        staff_first_name text, -- First name of the staff member
        staff_last_name text, -- Last name of the staff member
        other_staff_details text -- Other details about the staff member
    );
    CREATE TABLE IF NOT EXISTS problems (
        problem_id number, -- Unique identifier for the problem
        product_id number, -- Unique identifier for the product associated with the problem
        closure_authorised_by_staff_id number, -- Unique identifier for the staff member who authorised the closure of the problem
        reported_by_staff_id number, -- Unique identifier for the staff member who reported the problem
        date_problem_reported time, -- Date the problem was reported
        date_problem_closed time, -- Date the problem was closed
        problem_description text, -- Description of the problem
        other_problem_details text -- Other details about the problem
    );
    CREATE TABLE IF NOT EXISTS product (
        product_id number, -- Unique identifier for the product
        product_name text, -- Name of the product
        product_details text -- Details about the product
    );
    CREATE TABLE IF NOT EXISTS problem_log (
        problem_log_id number, -- Unique identifier for the problem log entry
        assigned_to_staff_id number, -- Unique identifier for the staff member assigned to the problem
        problem_id number, -- Unique identifier for the problem
        problem_category_code text, -- Code for the category of the problem
        problem_status_code text, -- Code for the status of the problem
        log_entry_date time, -- Date the log entry was made
        log_entry_description text, -- Description of the log entry
        log_entry_fix text, -- Fix for the problem described in the log entry
        other_log_details text -- Other details about the log entry
    );
    CREATE TABLE IF NOT EXISTS problem_status_codes (
        problem_status_code text, -- Code for the status of the problem
        problem_status_description text -- Description of the status of the problem
    );
    CREATE TABLE IF NOT EXISTS problem_category_codes (
        problem_category_code text, -- Code for the category of the problem
        problem_category_description text -- Description of the category of the problem
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS invoices (
        id VARCHAR, -- Unique identifier for the invoice
        customer_id VARCHAR -- Unique identifier for the customer associated with the invoice
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        id VARCHAR -- Unique identifier for the track
    );
    CREATE TABLE IF NOT EXISTS customers (
        id VARCHAR, -- Unique identifier for the customer
        first_name VARCHAR, -- First name of the customer
        last_name VARCHAR -- Last name of the customer
    );
    CREATE TABLE IF NOT EXISTS invoice_lines (
        track_id VARCHAR, -- Unique identifier for the track associated with the invoice line
        invoice_id VARCHAR -- Unique identifier for the invoice associated with the invoice line
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS employee (
        emp_fname VARCHAR, -- First name of the employee
        emp_num VARCHAR -- Unique identifier for the employee
    );
    CREATE TABLE IF NOT EXISTS CLASS (
        prof_num VARCHAR -- Unique identifier for the professor teaching the class
    );
    CREATE TABLE IF NOT EXISTS department (
        dept_code VARCHAR, -- Code for the department
        dept_name VARCHAR -- Name of the department
    );
    CREATE TABLE IF NOT EXISTS professor (
        emp_num VARCHAR, -- Unique identifier for the professor
        dept_code VARCHAR -- Code for the department the professor belongs to
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS course (
        crs_code text, -- Code for the course
        dept_code text, -- Code for the department offering the course
        crs_description text, -- Description of the course
        crs_credit number -- Number of credits for the course
    );
    CREATE TABLE IF NOT EXISTS class (
        class_code text, -- Code for the class
        crs_code text, -- Code for the course associated with the class
        class_section text, -- Section of the class
        class_time text, -- Time the class meets
        class_room text, -- Room the class meets in
        prof_num number -- Unique identifier for the professor teaching the class
    );
    CREATE TABLE IF NOT EXISTS employee (
        emp_num number, -- Unique identifier for the employee
        emp_lname text, -- Last name of the employee
        emp_fname text, -- First name of the employee
        emp_initial text, -- Initial of the employee's middle name
        emp_jobcode text, -- Code for the employee's job
        emp_hiredate time, -- Date the employee was hired
        emp_dob time -- Date of birth of the employee
    );
    CREATE TABLE IF NOT EXISTS professor (
        emp_num number, -- Unique identifier for the professor
        dept_code text, -- Code for the department the professor belongs to
        prof_office text, -- Office location of the professor
        prof_extension text, -- Extension number for the professor's phone
        prof_high_degree text -- Highest degree earned by the professor
    );
    CREATE TABLE IF NOT EXISTS student (
        stu_num number, -- Unique identifier for the student
        stu_lname text, -- Last name of the student
        stu_fname text, -- First name of the student
        stu_init text, -- Initial of the student's middle name
        stu_dob time, -- Date of birth of the student
        stu_hrs number, -- Number of hours the student has completed
        stu_class text, -- Classification of the student (e.g., freshman, sophomore)
        stu_gpa number, -- Grade point average of the student
        stu_transfer number, -- Number of transfer credits the student has
        dept_code text, -- Code for the department the student belongs to
        stu_phone text, -- Phone number of the student
        prof_num number -- Unique identifier for the professor who is the student's advisor
    );
    CREATE TABLE IF NOT EXISTS enroll (
        class_code text, -- Code for the class the student is enrolled in
        stu_num number, -- Unique identifier for the student
        enroll_grade text -- Grade the student received in the class
    );
    CREATE TABLE IF NOT EXISTS department (
        dept_code text, -- Code for the department
        dept_name text, -- Name of the department
        school_code text, -- Code for the school the department belongs to
        emp_num number, -- Unique identifier for the employee who is the head of the department
        dept_address text, -- Address of the department
        dept_extension text -- Extension number for the department's phone
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS playlists (
        id VARCHAR, -- Unique identifier for the playlist
        name VARCHAR -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        track_id VARCHAR, -- Unique identifier for the track
        playlist_id VARCHAR -- Unique identifier for the playlist the track is in
    );
    CREATE TABLE IF NOT EXISTS tracks (
        name VARCHAR, -- Name of the track
        id VARCHAR -- Unique identifier for the track
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS person (
        name text, -- Name of the person
        age number, -- Age of the person
        city text, -- City the person lives in
        gender text, -- Gender of the person
        job text -- Job of the person
    );
    CREATE TABLE IF NOT EXISTS personfriend (
        name text, -- Name of the person
        friend text, -- Name of the person's friend
        year number -- Year the person and friend became friends
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS affected_region (
        region_id VARCHAR, -- Unique identifier for the region
        storm_id VARCHAR -- Unique identifier for the storm that affected the region
    );
    CREATE TABLE IF NOT EXISTS storm (
        Name VARCHAR, -- Name of the storm
        storm_id VARCHAR -- Unique identifier for the storm
    );
    CREATE TABLE IF NOT EXISTS region (
        region_id VARCHAR, -- Unique identifier for the region
        Region_name VARCHAR -- Name of the region
    );
    '''
]
gpt_context_lvl4 += [
    '''
    CREATE TABLE IF NOT EXISTS Album (
        Title VARCHAR, -- Title of the album
        AlbumId VARCHAR -- Unique identifier for the album
    );
    CREATE TABLE IF NOT EXISTS Genre (
        GenreID VARCHAR, -- Unique identifier for the genre
        Name VARCHAR -- Name of the genre
    );
    CREATE TABLE IF NOT EXISTS Track (
        AlbumId VARCHAR, -- Unique identifier for the album the track is on
        GenreID VARCHAR -- Unique identifier for the genre of the track
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS projects (
        code text, -- Code for the project
        name text, -- Name of the project
        hours number -- Number of hours spent on the project
    );
    CREATE TABLE IF NOT EXISTS assignedto (
        scientist number, -- Unique identifier for the scientist assigned to the project
        project text -- Code for the project the scientist is assigned to
    );
    CREATE TABLE IF NOT EXISTS scientists (
        ssn number, -- Social security number of the scientist
        name text -- Name of the scientist
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS dorm (
        dormid number, -- Unique identifier for the dorm
        dorm_name text, -- Name of the dorm
        student_capacity number, -- Number of students the dorm can hold
        gender text -- Gender of the students who live in the dorm
    );
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid number, -- Unique identifier for the dorm with the amenity
        amenid number -- Unique identifier for the amenity
    );
    CREATE TABLE IF NOT EXISTS lives_in (
        stuid number, -- Unique identifier for the student living in the dorm
        dormid number, -- Unique identifier for the dorm the student lives in
        room_number number -- Room number of the student's dorm room
    );
    CREATE TABLE IF NOT EXISTS student (
        stuid number, -- Unique identifier for the student
        lname text, -- Last name of the student
        fname text, -- First name of the student
        age number, -- Age of the student
        sex text, -- Gender of the student
        major number, -- Major of the student
        advisor number, -- Unique identifier for the student's advisor
        city_code text -- Code for the city where the student is from
    );
    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid number, -- Unique identifier for the amenity
        amenity_name text -- Name of the amenity
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS has_amenity (
        dormid VARCHAR, -- Unique identifier for the dorm with the amenity
        amenid VARCHAR -- Unique identifier for the amenity
    );
    CREATE TABLE IF NOT EXISTS dorm_amenity (
        amenid VARCHAR, -- Unique identifier for the amenity
        amenity_name VARCHAR -- Name of the amenity
    );
    CREATE TABLE IF NOT EXISTS dorm (
        dorm_name VARCHAR, -- Name of the dorm
        dormid VARCHAR -- Unique identifier for the dorm
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS laptimes (
        raceid number, -- Unique identifier for the race
        driverid number, -- Unique identifier for the driver
        lap number, -- Lap number
        position number, -- Position of the driver in the race
        time text, -- Time of the lap
        milliseconds number -- Milliseconds of the lap time
    );
    CREATE TABLE IF NOT EXISTS constructors (
        constructorid number, -- Unique identifier for the constructor
        constructorref text, -- Reference for the constructor
        name text, -- Name of the constructor
        nationality text, -- Nationality of the constructor
        url text -- URL for the constructor
    );
    CREATE TABLE IF NOT EXISTS seasons (
        year number, -- Year of the season
        url text -- URL for the season
    );
    CREATE TABLE IF NOT EXISTS driverstandings (
        driverstandingsid number, -- Unique identifier for the driver standings
        raceid number, -- Unique identifier for the race
        driverid number, -- Unique identifier for the driver
        points number, -- Points earned by the driver
        position number, -- Position of the driver in the standings
        positiontext text, -- Position of the driver in text format
        wins number -- Number of wins by the driver
    );
    CREATE TABLE IF NOT EXISTS results (
        resultid number, -- Unique identifier for the result
        raceid number, -- Unique identifier for the race
        driverid number, -- Unique identifier for the driver
        constructorid number, -- Unique identifier for the constructor
        number number, -- Number of the driver's car
        grid number, -- Grid position of the driver
        position number, -- Position of the driver in the race
        positiontext text, -- Position of the driver in text format
        positionorder number, -- Position order of the driver
        points number, -- Points earned by the driver
        laps number, -- Number of laps completed by the driver
        time text, -- Time of the driver's race
        milliseconds number, -- Milliseconds of the driver's race time
        fastestlap number, -- Fastest lap of the driver
        rank number, -- Rank of the driver's fastest lap
        fastestlaptime text, -- Time of the driver's fastest lap
        fastestlapspeed text, -- Speed of the driver's fastest lap
        statusid number -- Unique identifier for the status of the driver's race
    );
    CREATE TABLE IF NOT EXISTS pitstops (
        raceid number, -- Unique identifier for the race
        driverid number, -- Unique identifier for the driver
        stop number, -- Stop number of the pit stop
        lap number, -- Lap number of the pit stop
        time text, -- Time of the pit stop
        duration text, -- Duration of the pit stop
        milliseconds number -- Milliseconds of the pit stop duration
    );
    CREATE TABLE IF NOT EXISTS status (
        statusid number, -- Unique identifier for the status
        status text -- Status of the driver's race
    );
    CREATE TABLE IF NOT EXISTS constructorstandings (
        constructorstandingsid number, -- Unique identifier for the constructor standings
        raceid number, -- Unique identifier for the race
        constructorid number, -- Unique identifier for the constructor
        points number, -- Points earned by the constructor
        position number, -- Position of the constructor in the standings
        positiontext text, -- Position of the constructor in text format
        wins number -- Number of wins by the constructor
    );
    CREATE TABLE IF NOT EXISTS circuits (
        circuitid number, -- Unique identifier for the circuit
        circuitref text, -- Reference for the circuit
        name text, -- Name of the circuit
        location text, -- Location of the circuit
        country text, -- Country of the circuit
        lat number, -- Latitude of the circuit
        lng number, -- Longitude of the circuit
        alt number, -- Altitude of the circuit
        url text -- URL for the circuit
    );
    CREATE TABLE IF NOT EXISTS qualifying (
        qualifyid number, -- Unique identifier for the qualifying session
        raceid number, -- Unique identifier for the race
        driverid number, -- Unique identifier for the driver
        constructorid number, -- Unique identifier for the constructor
        number number, -- Number of the driver's car
        position number, -- Position of the driver in the qualifying session
        q1 text, -- Time of the driver's Q1 session
        q2 text, -- Time of the driver's Q2 session
        q3 text -- Time of the driver's Q3 session
    );
    CREATE TABLE IF NOT EXISTS drivers (
        driverid number, -- Unique identifier for the driver
        driverref text, -- Reference for the driver
        number number, -- Number of the driver's car
        code text, -- Code for the driver
        forename text, -- First name of the driver
        surname text, -- Last name of the driver
        dob text, -- Date of birth of the driver
        nationality text, -- Nationality of the driver
        url text -- URL for the driver
    );
    CREATE TABLE IF NOT EXISTS races (
        raceid number, -- Unique identifier for the race
        year number, -- Year of the race
        round number, -- Round number of the race
        circuitid number, -- Unique identifier for the circuit
        name text, -- Name of the race
        date text, -- Date of the race
        time text, -- Time of the race
        url text -- URL for the race
    );
    CREATE TABLE IF NOT EXISTS constructorresults (
        constructorresultsid number, -- Unique identifier for the constructor results
        raceid number, -- Unique identifier for the race
        constructorid number, -- Unique identifier for the constructor
        points number, -- Points earned by the constructor
        status number -- Status of the constructor's result
    );
    '''
]
print(len(gpt_context_lvl1), len(gpt_context_lvl2), len(gpt_context_lvl3), len(gpt_context_lvl4))


# Actualizando el dataset

#### Paso 1: Agregando las columnas nuevas con datos generados al dataset como:
- "relations" para las relaciones entre tablas
- "context" para las tablas con comentarios

In [22]:
# mis columnas:
# "RELATIONS"        | "CONTEXT"
# gpt_relations_lvl1 | gpt_context_lvl1
# gpt_relations_lvl2 | gpt_context_lvl2
# gpt_relations_lvl3 | gpt_context_lvl3
# gpt_relations_lvl4 | gpt_context_lvl4

# Mis dataframes:
# lvl1_df
# lvl2_df
# lvl3_df
# lvl4_df

lvl1_df["relations"] = gpt_relations_lvl1
lvl2_df["relations"] = gpt_relations_lvl2
lvl3_df["relations"] = gpt_relations_lvl3
lvl4_df["relations"] = gpt_relations_lvl4

lvl1_df["context"] = context_lvl1
lvl2_df["context"] = gpt_context_lvl2
lvl3_df["context"] = gpt_context_lvl3
lvl4_df["context"] = gpt_context_lvl4


In [ ]:
lvl1_df = lvl1_df[-5:]
lvl2_df = lvl3_df[-5:]
lvl3_df = lvl3_df[-5:]
lvl4_df = lvl4_df[-5:]

lvl1_df["relations"] = relations_lvl1
lvl2_df["relations"] = relations_lvl2
lvl3_df["relations"] = relations_lvl3
lvl4_df["relations"] = relations_lvl4

lvl1_df["context"] = context_lvl1
lvl2_df["context"] = context_lvl2
lvl3_df["context"] = context_lvl3
lvl4_df["context"] = context_lvl4

#### Paso 2: Gabrando el dataset actualizado en un archivo de excel

In [45]:
# necesito: 
# A instruction = question 
# B response = answer 
# C relations 
# D context

from  train.mistral7bbase.functions.data_functions import write_data

i = 0
for _, row in lvl1_df.iterrows():
    write_data("train", ("A", i, row["instruction"]))
    write_data("train", ("B", i, row["response"]))
    write_data("train", ("C", i, row["relations"]))
    write_data("train", ("D", i, row["context"]))
    i += 1
i += 1
for _, row in lvl2_df.iterrows():
    write_data("train", ("A", i, row["instruction"]))
    write_data("train", ("B", i, row["response"]))
    write_data("train", ("C", i, row["relations"]))
    write_data("train", ("D", i, row["context"]))
    i += 1
i += 1
for _, row in lvl3_df.iterrows():
    write_data("train", ("A", i, row["instruction"]))
    write_data("train", ("B", i, row["response"]))
    write_data("train", ("C", i, row["relations"]))
    write_data("train", ("D", i, row["context"]))
    i += 1
i += 1
for _, row in lvl4_df.iterrows():
    write_data("train", ("A", i, row["instruction"]))
    write_data("train", ("B", i, row["response"]))
    write_data("train", ("C", i, row["relations"]))
    write_data("train", ("D", i, row["context"]))
    i += 1

Dato grabado exitosamente en la hoja train, celda A2.
Dato grabado exitosamente en la hoja train, celda B2.
Dato grabado exitosamente en la hoja train, celda C2.
Dato grabado exitosamente en la hoja train, celda D2.
Dato grabado exitosamente en la hoja train, celda A3.
Dato grabado exitosamente en la hoja train, celda B3.
Dato grabado exitosamente en la hoja train, celda C3.
Dato grabado exitosamente en la hoja train, celda D3.
Dato grabado exitosamente en la hoja train, celda A4.
Dato grabado exitosamente en la hoja train, celda B4.
Dato grabado exitosamente en la hoja train, celda C4.
Dato grabado exitosamente en la hoja train, celda D4.
Dato grabado exitosamente en la hoja train, celda A5.
Dato grabado exitosamente en la hoja train, celda B5.
Dato grabado exitosamente en la hoja train, celda C5.
Dato grabado exitosamente en la hoja train, celda D5.
Dato grabado exitosamente en la hoja train, celda A6.
Dato grabado exitosamente en la hoja train, celda B6.
Dato grabado exitosamente en

#### Paso 3: Agregando el prompt para entrenamiento del modelo

  Leemos la la hoja de excel

In [87]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")
from train.mistral7bbase.functions.data_functions import read_data

df = read_data("test")

In [88]:
# Importa pandas
import pandas as pd

# Supongamos que tienes un DataFrame llamado df
# Reemplaza df con el nombre real de tu DataFrame

# Encuentra la longitud de cada cadena en todas las columnas y encuentra el máximo
max_lengths = df.applymap(lambda x: len(str(x))).max()

# Encuentra el nombre de la columna con la longitud máxima
column_with_max_length = max_lengths[max_lengths == max_lengths.max()].index[0]

# Encuentra la fila con la cadena más larga en la columna con la longitud máxima
row_with_max_length = df.loc[df[column_with_max_length].astype(str).str.len().idxmax()]

# Imprime la fila con la cadena más larga
print("La fila con la cadena más larga en la columna '{}' es:".format(column_with_max_length))
print(row_with_max_length)

La fila con la cadena más larga en la columna 'texts' es:
question     find average temperature recorded in August 20...
answer       SELECT AVG(vd.Valor) AS Average_Temperature\nF...
relations    -- med_sistema_medicion.IdPlataforma_fk can be...
context      CREATE TABLE IF NOT EXISTS dbo_v2.med_sistema_...
texts        <s>Your task is to Generate a SQL code to the ...
Name: 20, dtype: object


C:\Users\lauth\AppData\Local\Temp\ipykernel_17120\2129580080.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_lengths = df.applymap(lambda x: len(str(x))).max()


In [91]:
len(row_with_max_length["texts"])

11473

Generamos el template

In [74]:
train_template = """<s>Your task is to Generate a SQL code to the question.
[INST]The query will run on a database with the following schema:
{context}

{relations}

### Answer
Given the database schema, here is the question: '''{question}'''[/INST]
{answer}</s>"""

test_template = """<s>Your task is to Generate a SQL code to the question.
[INST]The query will run on a database with the following schema:
{context}

{relations}

### Answer
Given the database schema, here is the question: '''{question}'''[/INST]
"""

Iteramos y agregamos una columna que tenga el template

In [77]:
from train.mistral7bbase.functions.data_functions import write_data

def write_texts(state):
    if state == "train":
        texts_template = train_template
    else:
        texts_template = test_template
        
    for index, row in data_df.iterrows():
        context = row["context"]
        question = row["question"]
        answer = row["answer"]
        relations = ""
        if type(row["relations"]) == str:
            relations = row["relations"]
            
        if state == "train":
            data = texts_template.format(context = context, relations = relations, question = question, answer = answer)
            write_data("train", ("E", index, data))
        else:
            data = texts_template.format(context = context, relations = relations, question = question)
            write_data("test", ("E", index, data))
			
			

In [78]:
write_texts("test")

Dato grabado exitosamente en la hoja test, celda E2.
Dato grabado exitosamente en la hoja test, celda E3.
Dato grabado exitosamente en la hoja test, celda E4.
Dato grabado exitosamente en la hoja test, celda E5.
Dato grabado exitosamente en la hoja test, celda E6.
Dato grabado exitosamente en la hoja test, celda E7.
Dato grabado exitosamente en la hoja test, celda E8.
Dato grabado exitosamente en la hoja test, celda E9.
Dato grabado exitosamente en la hoja test, celda E10.
Dato grabado exitosamente en la hoja test, celda E11.
Dato grabado exitosamente en la hoja test, celda E12.
Dato grabado exitosamente en la hoja test, celda E13.
Dato grabado exitosamente en la hoja test, celda E14.
Dato grabado exitosamente en la hoja test, celda E15.
Dato grabado exitosamente en la hoja test, celda E16.
Dato grabado exitosamente en la hoja test, celda E17.
Dato grabado exitosamente en la hoja test, celda E18.
Dato grabado exitosamente en la hoja test, celda E19.
Dato grabado exitosamente en la hoja